# Test des implémentations

Dans ce notebook, nous définissons les méthodes de test de notre algorithme de génération de colonnes.

## Vérification du fonctionnement du code

### Comparaison des solutions de (LP) par Bellman-Ford et PLNE

In [1]:
function verif_sol_BF_PLNE(instances)
    """
    Vérification que la résolution via Bellman-Ford et via PLNE rendent la même solution pour (LP).
    
    Entrée : 
        - instances : Array - Vecteur des indices des instances à utiliser
    """
    # On ne souhaite ici aucun affichage
    print_log = 0
    # Tolérance considérée
    e = 1e-6
    # Récupération des graphes de test
    graphs = load_graphs(instances);

    test_optim = []
    prop_cycles = []
    
    # Pour toute valeur de K possible
    for K in Array(2:6)
        # Pour chaque graphe 
        for graph in graphs
            # Résolution avec Bellman-Ford
            BF = heuristique_gencol(graph,K,"all2",e,"BF",print_log);
            # Résolution avec le PLNE
            PLNE = heuristique_gencol(graph,K,"all2",e,"PLNE",print_log);
            # Cycles solution
            c_BF = BF[2]
            c_PLNE = PLNE[2]
            # Proportion de cycles identiques
            prop = 0
            # incrémentation de la proportion de cycles identiques si tel est le cas
            for cycle in c_BF
                if test_cycles(cycle,c_PLNE)
                    prop += 1
                end
            end
            # Ajout des nouvelles données aux listes
            push!(test_optim,abs(BF[3][1]-PLNE[3][1]) < e)
            push!(prop_cycles,round(2*prop/(length(c_BF)+length(c_PLNE)),digits=3))
        end
    end
    # Affichages
    println("Proportion de solutions optimales identiques dans les cas testés : "*string(100*sum(test_optim)/20)*" %.")
    println("Proportion moyenne de cycles communs dans les cas testés : "*string(round(100*mean(prop_cycles),digits=2))*" %.")
end

verif_sol_BF_PLNE (generic function with 1 method)

### Comparaison de l'encadrement rendu et de la valeur optimale (K=2)

In [ ]:
function compare_2(instances)
    """
    Vérification que la solution optimale se trouve entre la solution heuristique et la solution de (LP) pour K=2.
    Le test est conduit l'initialisation "all2", et pour les deux méthodes de résolution du sous-problème
    
    Entrées : 
        - instances : Array - Indices des instances à tester
    """
    
    # On ne souhaite ici aucun affichage
    print_log = 0
    # Tolérance considérée
    e = 1e-6
    # Récupération des graphes de test
    graphs = load_graphs(instances);
    
    test_BF = []
    test_PLNE = []

    K = 2
    # Pour tous les graphes à tester
    for graph in graphs
        # Résolution avec Belman-Ford
        BF = heuristique_gencol(graph,K,"all2",e,"BF",print_log)[3];
        # Résolution avec PLNE
        PLNE = heuristique_gencol(graph,K,"all2",e,"PLNE",print_log)[3];
        # Résolution avec la formulation compacte
        exact = compact_cycle_2(graph,false)[1];
        
        # Ajout des valeurs aux listes
        push!(test_BF, BF[length(BF)] - e <= exact && exact <= BF[1] + e)
        push!(test_PLNE, PLNE[length(PLNE)] - e <= exact && exact <= PLNE[1] + e)

    end
    # Affichages
    println("Valeur optimale dans l'intervalle rendu ?")
    println("  - BF   : "*string(100*sum(test_BF)/length(graphs))*" % des cas testés.")
    println("  - PLNE : "*string(100*sum(test_PLNE)/length(graphs))*" % des cas testés.")
    
end

### Comparaison de l'encadrement rendu et de la valeur optimale (K quelconque)

In [ ]:
function compare_K(instances,K)
    """
    Vérification que la solution optimale se trouve entre la solution heuristique et la solution de (LP).
    Le test est conduit l'initialisation "all2", pour les deux méthodes de résolution du sous-problème, et pour une liste de K donnée.
    
    Entrées : 
        - instances : Array - Indices des instances à tester
        - K         : Array - Valeurs de K à utiliser
    """
    
    # On ne souhaite ici aucun affichage
    print_log = 0
    # Tolérance considérée
    e = 1e-6
    # Récupération des graphes de test
    graphs = load_graphs(instances);

    test_BF = []
    test_PLNE = []
    
    # Pour chaque valeur de k possible
    for k in K
        # Pour chaque graphe de test
        for graph in graphs
            # Résolution avec Bellman-Ford
            BF = heuristique_gencol(graph,k,"all2",e,"BF",print_log)[3];
            # Résolution avec le PLNE
            PLNE = heuristique_gencol(graph,k,"all2",e,"PLNE",print_log)[3];
            # Résolution avec la formulation compacte
            exact = compact_cycle_K(graph,k,false)[1];
            
            # Ajout des valeurs aux listes
            push!(test_BF, BF[length(BF)] - e <= exact && exact <= BF[1] + e)
            push!(test_PLNE, PLNE[length(PLNE)] - e <= exact && exact <= PLNE[1] + e)
        end
    end
    # Affichages
    println("Valeur optimale dans l'intervalle rendu ?")
    println("  - BF   : "*string(100*sum(test_BF)/(5*length(graphs)))*" % des cas testés.")
    println("  - PLNE : "*string(100*sum(test_PLNE)/(5*length(graphs)))*" % des cas testés.")
end

## Visualisation des améliorations faites

### Comparaison Bellman-Ford / PLNE

In [ ]:
function compare_BF_PLNE(N,K,densite)
    """
    Comparaison des temps de calcul avec Bf et PLNE, pour les n et K donnés.
    
    Entrées : 
        - N       : Array - Valeurs de n à tester
        - K       : Array - Valeurs de K à utiliser
        - densite : Float - Densité d'arcs dans le graphe
    """
    
    # Paramètres
    e = 1e-6;
    # On ne souhaite pas afficher les logs
    print_log = 0;

    # Matrices des temps de calcul pour chaque méthode
    T_BF = zeros(Float64, length(N), length(K))
    T_PLNE = zeros(Float64, length(N), length(K))

    # On remplit les matrices itérativement
    for (i,n) in enumerate(N)  
        # Nombre d'arcs dans les graphes de taille n
        m = round(Int,densite*n*(n-1))

        for (j,k) in enumerate(K)
            # On réalise 3 itérations
            for c in Array(1:3)
                graph = generation_graphe(n,m)
                # Pour tous les K possibles, on détermine le temps de calcule avec les deux méthodes de résolution
                T_BF[i,j] += 1/3 .* @elapsed model_BF = heuristique_gencol(graph,k,"all2",e,"BF",print_log)
                T_PLNE[i,j] += 1/3 .* @elapsed model_PLNE = heuristique_gencol(graph,k,"all2",e,"PLNE",print_log)
            end
        end
        #Affichage de l'avancement du calcul (barre d'avancement)
        progress = floor(Int, i / length(N) * 20)  
        barre = "■" ^ progress                  
        vide = " " ^ (20 - progress)            

        # Affiche la barre et efface la ligne précédente
        print("\r[" * barre * vide * "] "*string(i)*"/"*string(length(N)))  # \r pour revenir au début de la ligne
        flush(stdout)  
    end
    
    # Affichage des temps de calcul avec leur écart
    p1 = surface(N, K, T_BF', 
                 xlabel="n", ylabel="K", zlabel="Temps (secondes)", 
                 title="Comparaison des temps d'exécution",
                 legend=:topright, 
                 alpha=0.5, 
                 color=:red, label="BF", colorbar=false,titlefont = font(10))

    p2 = surface!(N, K, T_PLNE', 
                 alpha=0.5, 
                 color=:blue, label="PLNE", colorbar=false)

    delta_BF_PLNE = T_BF .- T_PLNE
    p2 = surface(N, K, delta_BF_PLNE', 
                 xlabel="n", ylabel="K", zlabel="Δt (secondes)", 
                 title="Ecart de temps d'exécution",
                 legend=:topright,  
                 alpha=0.5, 
                 color=:gray, colorbar=false,titlefont = font(10))

    # Affichage des surfaces
    plot(p1, p2, layout=(1, 2))  
end

### Comparaison de la méthode d'initialisation

In [ ]:
function compare_init(N, K, densite)
    """
    Comparaison des méthodes d'initialisation, avec la résolution par PLNE.
    
    Entrées : 
        - N       : Array - Valeurs de n à tester
        - K       : Array - Valeurs de K à utiliser
        - densite : Float - Densité d'arcs dans le graphe
    """
    
    # Paramètres
    e = 1e-6;
    
    # On ne souhaite pas afficher les logs
    print_log = 0;

    # Grille répertoriant les écarts de temps
    T_2 = zeros(Float64, length(N), length(K))
    T_all2 = zeros(Float64, length(N), length(K))

    for (i,n) in enumerate(N)
        # Nombre d'arcs dans le graphe
        m = round(Int,densite*n*(n-1))
        
        for (j,k) in enumerate(K)
            # On moyenne le temps de calcul sur 3 instances
            for c in Array(1:3)
                graph = generation_graphe(n,m)

                T_2[i,j] += 1/3 .* @elapsed model = heuristique_gencol(graph,k,"2",e,"PLNE",print_log)
                T_all2[i,j] += 1/3 .* @elapsed model = heuristique_gencol(graph,k,"all2",e,"PLNE",print_log)
            end
        end
        # Affichage de l'avancement du code
        progress = floor(Int, i / length(N) * 20)  
        barre = "■" ^ progress                  
        vide = " " ^ (20 - progress)            

        # Affiche la barre et efface la ligne précédente
        print("\r[" * barre * vide * "] "*string(i)*"/"*string(length(N)))  # \r pour revenir au début de la ligne
        flush(stdout)  

    end

    # Affichage des temps de calcul et de leur écart 
    p1 = surface(N, K, T_2', 
                 xlabel="n", ylabel="K", zlabel="Temps (secondes)", 
                 title="Comparaison des temps d'exécution",
                 legend=:topright, 
                 alpha=0.5, 
                 color=:red, label="2", colorbar=false,titlefont = font(10))

    p2 = surface!(N, K, T_all2', 
                 alpha=0.5, 
                 color=:blue, label="all2", colorbar=false)

    delta_2_all2 = T_2 .- T_all2
    p2 = surface(N, K, delta_2_all2', 
                 xlabel="n", ylabel="K", zlabel="Δt (secondes)", 
                 title="Ecart de temps d'exécution",
                 legend=:topright,  
                 alpha=0.5, 
                 color=:gray, colorbar=false,titlefont = font(10))

    # Affichage des surfaces
    plot(p1, p2, layout=(1, 2))  

end

## Test des limites de l'algorithme

### Limites pour n et K variables

In [ ]:
function test_limit_N_K(N, K, densite)
    """
    Evalue le temps de calcul pour n et K donnés, avec l'initialisation "all2" et la résolution par PLNE.
    
    Entrées : 
        - N       : Array - Valeurs de n à tester
        - K       : Array - Valeurs de K à utiliser
        - densite : Float - Densité d'arcs dans le graphe
    """
    
    # Paramètres
    e = 1e-6;
    
    # On n'affiche pas les logs
    print_log = 0;

    # Matrice des temps de calcul
    T = zeros(Float64, length(N), length(K))
    for (i,n) in enumerate(N)
        # Nombre d'arcs dans le graphe
        m = round(Int,densite*n*(n-1))
        
        for (j,k) in enumerate(K)
            # On moyenne les temps de calcul sur 3 instances
            for c in Array(1:3)
                graph = generation_graphe(n,m)
                T[i,j] += 1/3 .* @elapsed model = heuristique_gencol(graph,k,"all2",e,"PLNE",print_log)
            end
        end
        # Affichage de l'avancement du code
        progress = floor(Int, i / length(N) * 20)  
        barre = "■" ^ progress                  
        vide = " " ^ (20 - progress)            

        # Affiche la barre et efface la ligne précédente
        print("\r[" * barre * vide * "] "*string(i)*"/"*string(length(N)))  # \r pour revenir au début de la ligne
        flush(stdout)  
    end

    # Affichage de la surface
    surface(K, N, T, xlabel="K", ylabel="n", zlabel="t (s)",
               title="Temps de calcul en fonction de n et K", legend=false,titlefont = font(10))
end

### Limites pour n variable, à K fixé

In [ ]:
function test_limit_N(N, K, densite)
    """
    Evalue le temps de calcul pour n donnés, et K fixé donné, avec l'initialisation "all2" et la version PLNE pour le sous-problème.
    
    Entrées : 
        - N       : Array   - Valeurs de n à tester
        - K       : Integer - Valeur de K à utiliser
        - densite : Float   - Densité d'arcs dans le graphe
    """
    # Paramètres
    e = 1e-6;
    # On n'affiche pas les logs
    print_log = 0;
    # Matrice des temps de calcul
    T = zeros(Float64, length(N))

    for (i,n) in enumerate(N)
        # Nombre d'arcs dans le graphe
        m = round(Int,densite*n*(n-1))
        # On moyenne les temps de calcul sur 3 instances
        for c in Array(1:3)
            graph = generation_graphe(n,m)
            T[i] += 1/3 .* @elapsed model = heuristique_gencol(graph,K,"all2",e,"PLNE",print_log)
        end
        # Affichage de l'avancement du code
        progress = floor(Int, i / length(N) * 20)  
        barre = "■" ^ progress                  
        vide = " " ^ (20 - progress)            

        # Affiche la barre et efface la ligne précédente
        print("\r[" * barre * vide * "] "*string(i)*"/"*string(length(N)))  # \r pour revenir au début de la ligne
        flush(stdout) 
    end 
    
    plot(N,T,size=(800, 600))
    ylabel!("Temps (s)")
    title!("Temps d'éxecution de notre meilleur algorithme en fonction de la taille des données",titlefont = font(10))

end

## Questionnement de la méthode implémentée

### Fonctions d'affichage

In [1]:
function print_tuple(arc,liste)
    """
    Fonction qui affiche un tuple, avec les sommets en rouge s'ils n'appartiennent pas à une liste donnée.
    
    Entrées :
        - arc   : Tuple - Tuple à afficher
        - liste : Array - Liste de sommets
    """
    # Affichage initial
    print("(")
    # Si le sommet n'est pas dans la liste, on l'affiche en rouge
    if !(arc[1] in liste)
        printstyled(arc[1];color = :red)
    else 
        # Sinon, on l'affiche en noir
        print(arc[1])
    end
    print(",")
    # Si le sommet n'appartient pas à la liste, on l'affiche en rouge
    if !(arc[2] in liste)
        printstyled(arc[2];color = :red)
    else 
        # Sinon on l'affiche en noir
        print(arc[2])
    end
    # Affichage final
    print(")")
        
end

print_tuple (generic function with 1 method)

In [ ]:
function print_cycle(cycle,liste)
    """
    Fonction d'affichage d'un cycle, en utilisant la fonction précédente.
    
    Entrées : 
        - cycle : Array - Liste de cycles à afficher
        - liste : Array - Liste de sommets, pour tester l'appartenance de chaque sommet du cycle
    """
    # Affichage initial
    print("[")
    # Pour chaque arc du cycle
    for (k,arc) in enumerate(cycle)
        # On affiche le tuple
        print_tuple(arc,liste)
        # Si on n'est pas arrivé au dernier arc, on ajoute une virgule
        if k < length(cycle)
            print(",")
        end
    end
    # Affichage final
    println("]")
end

### Étude et comparaison des patients greffés ou non

In [ ]:
function proportion_sommets_solution(K, indexes)
    """
    Calcule la proportion des sommets d'un graphe présents dans les cycles solution pour différentes instances et valeurs de K
    
    Entrées :
        - K       : Array - Valeurs de K à utiliser
        - indexes : Array - Indexes des instances à utiliser
    """
    # Paramètres
    print_log = 0
    e = 1e-6

    # Chargement des instances
    graphs = load_graphs(indexes);

    # Matrice contenant la proportion de sommets utilisés dans les cycles solution pour chaque instance et chaque K
    prop_sommets = zeros(Float64, length(graphs), length(K))

    for (j,k) in enumerate(K)
        for (i,graph) in enumerate(graphs)
            # Résolution de l'algorithme de génération de colonne
            res = heuristique_gencol(graph,k,"all2",e,"PLNE",print_log);
            
            # Récupération des cycles solution et de la taille du graphe
            cycles = res[2]
            n = graph[1]

            # Parcours des cycles pour récupérer tous les sommets présents
            sommets = Set()
            for cycle in cycles
                for (u,v) in cycle
                    push!(sommets, u)
                end
            end

            # Calcul de la porportion de sommets présents dans les cycles solution
            prop_sommets[i,j] = length(sommets)/n
        end
    end

    println("Proportion moyenne de sommets présents dans les cycles solution : ",round(100*mean(prop_sommets),digits=2),"%")
    println("Proportion minimum de sommets présents dans les cycles solution : ",round(100*minimum(prop_sommets),digits=2),"%")
    println("Proportion maximum de sommets présents dans les cycles solution : ",round(100*maximum(prop_sommets),digits=2),"%")
    surface(K, indexes, prop_sommets, xlabel="K", ylabel="instance", titlefont=font(10), title="Proportion de sommets utilisés en fonction de n et K")
end

In [ ]:
function difference_liaison_sommets(K, indexes)
    """
    Calcule, sur différentes instances, la densité de liaison moyenne des sommets présents ou non dans les cycles optimaux
    
    Entrées :
        - K       : Array - Valeurs de K à utiliser
        - indexes : Array - Indexes des instances à utiliser
    """
    # Paramètres
    print_log = 0
    e = 1e-6

    # Chargement des instances
    graphs = load_graphs(indexes)

    # Vecteurs contenant, pour chaque instance, la densité de liaison moyenne des sommets utilisés ou non dans les cycles optimaux
    moy_liens_s_optimaux = []
    moy_liens_s_non_optimaux = []

    for (i,graph) in enumerate(graphs)
        # Résolution de l'algorithme de génération de colonne
        res = heuristique_gencol(graph,K,"all2",e,"PLNE",print_log);

        # Récupération des cycles solution
        cycles = res[2]
        # Récupération des cractéristiques du graphe considéré
        n = graph[1]
        pred = graph[4]
        succ = graph[5]
        
        # Parcours des cycles pour récupérer les sommets optimaux et leur proportion de liens associée
        sommets_optimaux = Set()
        liens_sommets_optimaux = []
        for cycle in cycles
            for (u,v) in cycle
                push!(sommets_optimaux, u)
                push!(liens_sommets_optimaux,0.5*(length(pred[u])+length(succ[u]))/(n-1))
            end
        end
        
        # Récupération des sommets non présents dans les cycles optimaux
        sommets_non_optimaux = setdiff(Set(1:n),sommets_optimaux)
        # Parcours de ces sommets pour récupérer leur proportion de liens associée
        liens_sommets_non_optimaux = []
        for s in sommets_non_optimaux
            push!(liens_sommets_non_optimaux,0.5*(length(pred[s])+length(succ[s]))/(n-1))
        end
        
        push!(moy_liens_s_optimaux,mean(liens_sommets_optimaux))
        push!(moy_liens_s_non_optimaux,mean(liens_sommets_non_optimaux))
    end

    println("Densité de liaison moyenne pour les sommets présents dans les cycles optimaux : ",round(mean(moy_liens_s_optimaux),digits=3))
    println("Densité de liaison moyenne pour les sommets non présents dans les cycles optimaux : ",round(mean(moy_liens_s_non_optimaux),digits=3))
end

In [ ]:
function sommets_absents_C_hat(K, index)
    """
    Calcule le pourcentage de sommets absents de C_hat (final), et leur proportion d'arcs sortants et/ou entrants
    
    Entrées :
        - K       : Integer - Valeur de K à utiliser
        - index   : String - Index de l'instance de test
    """
    # Paramètres
    print_log = 0
    e = 1e-6

    # Récupération de l'instance et de ses caractéristiques
    graph = load_graphs([index])[1]
    n = graph[1]
    A = graph[2]
    pred = graph[4]
    succ = graph[5]

    # Résolution de l'algorithme de génération de colonne
    res = heuristique_gencol(graph,K,"all2",e,"PLNE",print_log);

    # Calcul du nombre de cycles par sommet dans C_hat à l'aide C_hat_v
    C_hat_v = res[5]
    nb_cycles_v = map(length, C_hat_v)
    # Récupération des sommets qui n'ont aucun cycle dans C_hat
    sommets = findall(x -> x == 0, nb_cycles_v)

    # Pour les sommets absents de C_hat, calcul des proportion d'arcs entrants et/ou sortants associées
    prop_totale = mean([0.5*(length(pred[u])+length(succ[u]))/(n-1) for u in sommets])
    prop_preds = mean([length(pred[u])/(n-1) for u in sommets])
    prop_succ = mean([length(succ[u])/(n-1) for u in sommets])

    println("Densité du graphe : ", round(length(A)/(n*(n-1)),digits=3))
    println("Pourcentage de sommets absents de C_hat : ", round(100*length(sommets)/n,digits=1)," %")
    println("- Pour les sommets absents de C_hat -")
    println("Proportion d'arcs par sommet moyenne : ",round(prop_totale,digits=3))
    println("Proportion d'arcs entrants moyenne : ",round(prop_preds,digits=3))
    println("Proportion d'arcs sortants moyenne : ",round(prop_succ,digits=3))
end

### Variation de la solution en fonction du choix de résolution du sous-problème

In [ ]:
function sommets_BF_vs_PLNE(K, indexes)
    """
    Etudie la différence dans les sommets solution entre une résolution du sous-problème avec BF ou un PLNE
    
    Entrées :
        - K       : Array - Valeurs de K à utiliser
        - indexes : Array - Indexes des instances de test
    """
    # Paramètres
    print_log = 0
    e = 1e-6

    # Chargement des instances de test
    graphs = load_graphs(indexes);

    # Initialisation des vecteur contenant la proportion de sommets communs et le nombre de sommets différentes moyens pour toutes les valeurs de K
    prop_sommets = []
    diff_nb_sommets = []

    for k in K
        # Proportions de sommets communs pour un K donné, pour chaque instance de test
        prop_sommets_K = []
        # Nombres de sommets avec BF et PLNE pour un K donné, pour chaque instance de test
        nb_sommets_BF_K = []
        nb_sommets_PLNE_K = []
        for graph in graphs
            # Résolution de l'algorithme avec le sous-problème BF ou PLNE
            BF = heuristique_gencol(graph,k,"all2",e,"BF",print_log);
            PLNE = heuristique_gencol(graph,k,"all2",e,"PLNE",print_log);
            
            # Récupération des cycles optimaux issues des résolutions BF et PLNE
            c_BF = BF[2]
            c_PLNE = PLNE[2]

            # Parcours des cycles issus de BF pour récupérer les sommets
            sommets_BF = Set()
            for cycle in c_BF
                for (i,j) in cycle
                    push!(sommets_BF, i)
                end
            end
            
            # Parcours des cycles issus de PLNE pour récupérer les sommets
            sommets_PLNE = Set()
            for cycle in c_PLNE
                for (i,j) in cycle 
                    push!(sommets_PLNE, i)
                end
            end

            # Calcul du nombre de sommets communs
            nb_sommets_communs = length(intersect(sommets_BF,sommets_PLNE))

            push!(nb_sommets_BF_K,length(sommets_BF))
            push!(nb_sommets_PLNE_K,length(sommets_PLNE))
            push!(prop_sommets_K,round(2*nb_sommets_communs/(length(sommets_BF)+length(sommets_PLNE)),digits=3))
        end
        push!(diff_nb_sommets,mean(abs.(nb_sommets_BF_K .- nb_sommets_PLNE_K)))
        push!(prop_sommets, mean(prop_sommets_K))
    end

    println("Différence moyenne du nombre de sommets dans les solutions rendues avec BF ou PLNE : ",round(mean(diff_nb_sommets),digits=3))
    println("Proportion de sommets communs parmi les sommets des cycles optimaux : ",round(mean(prop_sommets),digits=3))
    println("Cas de K = 3")
    println("- Différence moyenne du nombre de sommets : ", diff_nb_sommets[2])
    println("- Proportion moyenne de sommets communs : ", round(prop_sommets[2],digits=3))
end

### Renumérotation des sommets du graphe

In [ ]:
function encode(graph)
    """
    Fonction qui change la numérotation des sommets d'un graphe, via une clé bijective aléatoire.
    
    Entrées :
        - graph : Array - Vecteur des propriétés du graphe.
    
    Sorties :
        - new_graph : Array - Vecteur des propriétés du graphe renuméroté
        - encoder   : Array - Clé de renumérotation utilisée
    """
    # Récupération des caractéristiques du graphe initial
    A = graph[2]
    n = graph[1]
    # Création de la clé de renumérotation
    encoder = shuffle!(Array(1:n))
    
    # Initialisation des propriétés du nouveau graphe
    new_A = []
    new_W = zeros(Float64,n,n)
    new_preds = [[] for i in range(1,n)]
    new_succ = [[] for i in range(1,n)]
    # Pour chaque arc, on met à jour le nouveau graphe, en tenant compte de la nouvelle numérotation
    for arc in A
        new_arc = (encoder[arc[1]],encoder[arc[2]])
        push!(new_A, new_arc)
        push!(new_preds[encoder[arc[2]]],encoder[arc[1]])
        push!(new_succ[encoder[arc[1]]],encoder[arc[2]])
        new_W[encoder[arc[1]],encoder[arc[2]]] = 1.0
    end
    # Création du vecteur de propriétés
    new_graph = [n,new_A,new_W,new_preds,new_succ]
    
    return new_graph, encoder
end

In [ ]:
function decode(encoder, cycles)
    """
    Fonction qui décode des cycles solutions, en inversant la clé de renumérotation.
    
    Entrées :
        - encoder : Array - Clé de renumérotation utilisée
        - cycles  : Array - Liste de cycles 
    
    Sorties :
        - new_cycles : Array - Liste des cycles en inversant la numérotation réalisée
    """
    # Nombre de sommets
    n = length(encoder)
    # Initialisation du décodeur
    decoder = Array(1:n)
    # Construction de la clé inverse itérativement
    for i in 1:n
        decoder[encoder[i]] = i
    end
    # Construction des nouveaux cycles
    new_cycles = []
    for cycle in cycles
        new_cycle = []
        # Pour chaque arc, on change la numérotation, et on ajoute le nouvel arc au cycle
        for arc in cycle
            new_arc = (decoder[arc[1]],decoder[arc[2]])
            push!(new_cycle, new_arc)
        end
        # Ajout du cycle renuméroté dans la liste
        push!(new_cycles,new_cycle)
    end
    return new_cycles
end

In [ ]:
function test_renumerotation(instance, K, method)
    """
    Fonction de test d'une méthode donnée, en renumérotant un graphe initial.
    
    Entrées :
        - instance : String  - Indice de l'instance de test souhaitée
        - K        : Integer - Valeur de K souhaitée
        - method   : String  - Méthode de résolution du sous-problème
    """
    
    # Paramètres de la résolution
    nodes = Array(1:n)
    e = 1e-6;
    print_log = 0;

    # Récupération du graphe souhaité
    graph = load_graphs(instance)[1];
    
    # Résolution avec la numérotation classique
    numerotation_1 = heuristique_gencol(graph,K,"all2",e,method,print_log); 

    # Renumérotation du graphe
    new_graph,encoder = encode(graph)
    # Résolution du problème
    numerotation_2 = heuristique_gencol(new_graph,K,"all2",e,method,print_log);
    # Décodage des cycles obtenus
    new_cycles = decode(encoder, numerotation_2[2])
    
    # Récupération de la liste de sommets des cycles solutions (numerotation n°1)
    arcs_1 = vcat(numerotation_1[2]...)
    liste_sommets_1 = []
    for arc in arcs_1
        push!(liste_sommets_1,arc[1])
    end
    # Récupération de la liste de sommets des cycles solutions (numerotation n°2)
    arcs_2 = vcat(new_cycles...)
    liste_sommets_2 = []
    for arc in arcs_2
        push!(liste_sommets_2,arc[1])
    end
    
    # Affichage des cycles pour la première numérotation
    println("Numérotation n°1 :")
    for cycle in numerotation_1[2]
        print(" - ")
        if test_cycles(cycle,new_cycles)
            printstyled(cycle;color = :green)
            print("\n")
        else
            print_cycle(cycle,liste_sommets_2)
        end
    end
    println("\n")
    # Affichage des cycles pour la seconde numérotation
    println("Numérotation n°2 : ")
    for cycle in new_cycles
        print(" - ")
        if test_cycles(cycle,numerotation_1[2])
            printstyled(cycle;color = :green)
            print("\n")
        else
            print_cycle(cycle,liste_sommets_1)
        end
    end 
end

### Graphe avec individu urgent et rare

In [ ]:
function show_cycle_ethic(cycles,individu)
    """
    Fonction d'affichage d'une liste de cycles, en affichant un sommet donnée en rouge.
    
    Entrées :
        - cycles   : Array   - Liste de cycles à afficher
        - individu : Integer - Sommet à afficher en rouge
    """
    # Pour chaque cycle de la liste
    for cycle in cycles
        # Affichage initial
        print(" - ")
        print("[")
        # Affichage de chaque arc
        for (k,arc) in enumerate(cycle)
            print("(")
            if (arc[1] in [individu])
                printstyled(arc[1];color = :red)
            else 
                print(arc[1])
            end
            print(",")
            if (arc[2] in [individu])
                printstyled(arc[2];color = :red)
            else 
                print(arc[2])
            end
            print(")")
            if k < length(cycle)
                print(",")
            end
        end
        # Affichage final
        println("]")

    end 
end

In [ ]:
function is_in_sol(individu,cycles)
    """
    Teste si l'individu donné est dans la liste de cycles.
    
    Entrées :
        - individu : Integer - Indice de l'individu d'intérêt
        - cycles   : Array   - Liste des cycles à tester
    
    Sortie : 
        - Booléen - Résultat du test d'appartenance de l'individu aux cycles
    
    """
    arcs = vcat(cycles...)
    sommets = []
    
    for (i,j) in arcs
        push!(sommets,i)
    end
    
    return (individu in sommets)
end

In [ ]:
function generate_ind_urgent(instance,individu,prop)
    """
    Crée un graphe avec un individu rare, à partir de l'instance donnée.
    
    Entrées : 
        - instance : String  - Nom de l'instance initiale
        - individu : Integer - Indice de l'individu rare
        - prop     : Float   - Proportion d'arcs à supprimer
    
    Sortie :
        - graph : Array - Graphe modifié
    """
    
    # Génération d'un graphe aléatoire, et première résolution
    graph = load_graphs(instance)[1];
    n = graph[1]
    # On sélectionne un individu

    # On fixe la graine pour la reproductibilité
    Random.seed!(4)
    # On détermine les arcs passant par l'individu
    succ_individu = intersect([(individu, x) for x in 1:n if x != individu],graph[2])
    preds_individu = intersect([(x, individu) for x in 1:n if x != individu],graph[2])

    # On retire prop% des prédécesseurs et prop% des successeurs
    num_s = Int(floor(prop * length(succ_individu)))
    indices_a_retirer = randperm(length(succ_individu))[1:num_s]
    succ_a_retirer = [succ_individu[i] for i in indices_a_retirer]

    num_p = Int(floor(prop * length(preds_individu)))
    indices_a_retirer = randperm(length(preds_individu))[1:num_p]
    preds_a_retirer = [preds_individu[i] for i in indices_a_retirer]

    arcs_a_retirer = vcat(succ_a_retirer,preds_a_retirer)

    # Création du nouveau graphe
    W = graph[3]
    preds = graph[4]
    succ = graph[5]

    # On modifie A, W, preds et succ
    for (i,j) in arcs_a_retirer
        W[i,j] = 0.0
        # On retire i dans les prédécesseurs de j
        preds[j] = filter(x -> x != i,graph[4][j])
        # On retire j dans les successeurs de i
        succ[i] = filter(x -> x != j,graph[5][i])
    end
    # Mise à jour des arcs
    A = filter(x -> !(x in arcs_a_retirer), graph[2])

    graph = [n,A,W,preds,succ]
    
    return graph
end

### Augmentation du poids des patients les plus rares

In [ ]:
function augmentation_patients_rares(K, instance)
    """
    Résoud la génération de colonne normalement puis en augmentant le poids des 10% de patients ayant le moins de prédecesseurs
    
    Entrées :
        - K        : Integer - Valeur de K à utiliser
        - instance : String - Index de l'instance de test
    """
    # Paramètres
    print_log = 0
    e = 1e-6

    # Chargement du graphe de test et récupération de ses propriétés
    graph = load_graphs([instance])[1]
    n = graph[1]
    A = graph[2]
    W = graph[3]
    pred = graph[4]
    succ = graph[5]

    # Calcul du nombre de prédecesseurs de chaque sommet
    nb_preds_sommets = map(length, pred)

    # Nombre de patients à qui augmenter le poids
    nb = round(Int64,ceil(0.1*n))
    # Récupération des nb patients à qui augmenter le poids
    sommets_a_augmenter = partialsortperm(nb_preds_sommets, 1:nb)
    println("Nombre de patients à qui augmenter le poids (les 10% ayant le moins de prédecesseurs) : ", nb)

    # Résolution de l'algorithme avec le même poids pour tout le monde
    res = heuristique_gencol(graph,K,"all2",e,"PLNE",print_log);
    cycles = res[2]

    # Calcul du nombre de sommets dans les sommets à augmenter qui sont dans la solution rendue
    nb_sommets_sol = 0
    for s in sommets_a_augmenter
        nb_sommets_sol += 1*is_in_sol(s,cycles)
    end
    println("- Avant augmentation (poids de 1) -")
    println("Nombres de sommets parmi les 10% avec le moins de prédécesseurs dans la solution : ", nb_sommets_sol)

    # Création d'une nouvelle matrice de poids où l'on augmente de 1 tous les arcs vers les sommets à augmenter
    W2 = copy(W)
    for s in sommets_a_augmenter
        for u in pred[s]
            W2[u,s] = 2
        end
    end
    # Résolution de l'algorithme avec les poids augmenter pour les patients ayant le moins de prédecesseurs
    graph2 = [n, A, W2, pred, succ]
    res2 = heuristique_gencol(graph2,K,"all2",e,"PLNE",print_log);
    cycles2 = res2[2]

    # Calcul du nombre de sommets dans les sommets à augmenter qui sont dans la solution rendue
    nb_sommets_sol2 = 0
    for s in sommets_a_augmenter
        nb_sommets_sol2 += 1*is_in_sol(s,cycles2)
    end
    println("- Après augmentation (poids de 2) -")
    println("Nombres de sommets parmi les 10% avec le moins de prédécesseurs dans la solution : ", nb_sommets_sol2)
end

In [ ]:
println("Fonctions de test importées avec succès.")